<a href="https://colab.research.google.com/github/imadegunawinangun/PrediksiSuhuDenpasar/blob/main/Prediksi_Suhu_Denpasar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
import os
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None
import random
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from scipy import stats

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


In [49]:
!git clone https://github.com/imadegunawinangun/PrediksiSuhuDenpasar

fatal: destination path 'PrediksiSuhuDenpasar' already exists and is not an empty directory.


In [50]:
ls PrediksiSuhuDenpasar/

openweatherdata-denpasar-1990-2020.csv      Prediksi_Suhu_Denpasar.ipynb
openweatherdata-denpasar-1990-2020v0.1.csv


In [51]:
data = pd.read_csv('PrediksiSuhuDenpasar/openweatherdata-denpasar-1990-2020.csv')
data.head()

,dt,dt_iso,timezone,city_name,lat,lon,temp,temp_min,temp_max,pressure,sea_level,grnd_level,humidity,wind_speed,wind_deg,rain_1h,rain_3h,rain_6h,rain_12h,rain_24h,rain_today,snow_1h,snow_3h,snow_6h,snow_12h,snow_24h,snow_today,clouds_all,weather_id,weather_main,weather_description,weather_icon
0,631152000,1990-01-01 00:00:00 +0000 UTC,28800,Denpasar,-8.652497,115.219117,25.82,25.82,25.82,1010.0,NaN,NaN,86,1.36,225,0.49,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,98,500,Rain,light rain,10d
1,631155600,1990-01-01 01:00:00 +0000 UTC,28800,Denpasar,-8.652497,115.219117,26.20,26.20,26.20,1011.0,NaN,NaN,84,2.09,247,0.13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,91,500,Rain,light rain,10d
2,631159200,1990-01-01 02:00:00 +0000 UTC,28800,Denpasar,-8.652497,115.219117,26.45,26.45,26.45,1011.0,NaN,NaN,84,2.44,262,0.24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,94,500,Rain,light rain,10d
3,631162800,1990-01-01 03:00:00 +0000 UTC,28800,Denpasar,-8.652497,115.219117,26.80,26.80,26.80,1011.0,NaN,NaN,82,2.29,271,0.23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,94,500,Rain,light rain,10d
4,631166400,1990-01-01 04:00:00 +0000 UTC,28800,Denpasar,-8.652497,115.219117,27.04,27.04,27.04,1010.0,NaN,NaN,82,1.71,274,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,76,803,Clouds,broken clouds,04d


In [52]:
data.tail()

,dt,dt_iso,timezone,city_name,lat,lon,temp,temp_min,temp_max,pressure,sea_level,grnd_level,humidity,wind_speed,wind_deg,rain_1h,rain_3h,rain_6h,rain_12h,rain_24h,rain_today,snow_1h,snow_3h,snow_6h,snow_12h,snow_24h,snow_today,clouds_all,weather_id,weather_main,weather_description,weather_icon
264919,1578423600,2020-01-07 19:00:00 +0000 UTC,28800,Denpasar,-8.652497,115.219117,27.0,27.0,27.0,1007.0,NaN,NaN,94,4.1,300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75,803,Clouds,broken clouds,04n
264920,1578427200,2020-01-07 20:00:00 +0000 UTC,28800,Denpasar,-8.652497,115.219117,27.0,27.0,27.0,1007.0,NaN,NaN,94,5.7,300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75,803,Clouds,broken clouds,04n
264921,1578430800,2020-01-07 21:00:00 +0000 UTC,28800,Denpasar,-8.652497,115.219117,28.0,28.0,28.0,1007.0,NaN,NaN,83,6.7,290,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75,803,Clouds,broken clouds,04n
264922,1578434400,2020-01-07 22:00:00 +0000 UTC,28800,Denpasar,-8.652497,115.219117,28.0,28.0,28.0,1007.0,NaN,NaN,83,6.2,290,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40,802,Clouds,scattered clouds,03n
264923,1578438000,2020-01-07 23:00:00 +0000 UTC,28800,Denpasar,-8.652497,115.219117,28.0,28.0,28.0,1008.0,NaN,NaN,83,5.7,300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40,802,Clouds,scattered clouds,03d


In [ ]:
data = pd.read_csv('PrediksiSuhuDenpasar/openweatherdata-denpasar-1990-2020.csv', 
                   low_memory=False
                   )

data.drop(["dt", "timezone", 'city_name', 'lat', 'lon', 'sea_level','grnd_level', 'rain_1h', 'rain_3h','rain_6h','rain_12h','rain_24h','rain_today','snow_1h','snow_3h','snow_6h','snow_12h','snow_24h','snow_today'], inplace=True, axis=1)
data.drop(['pressure','humidity','wind_speed','wind_deg','clouds_all', 'weather_id', 'weather_main', 'weather_description', 'weather_icon'], inplace=True, axis=1)

In [ ]:
data

In [ ]:
data.isnull().sum()


In [ ]:
data.set_index('dt_iso', inplace=True)

In [ ]:
plt.xlabel("Date")
plt.ylabel("Temperature")
plt.plot(data.temp)
plt.title('Temperature average',
          fontsize=20);

In [ ]:
data['temp']

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))
data=scaler.fit_transform(data)
data

In [ ]:
train = int(len(data)*0.8)
valid = len(data)-train

In [ ]:
train_data, valid_data = data[0:train,:], data[train:len(data),:1]

In [ ]:
def create_dataset(dataset, time_step=1):
  dataX, dataY = [],[]
  for i in range(len(dataset)-time_step-1):
    _data = dataset[i:(i+time_step), 0]
    dataX.append(_data)
    dataY.append(dataset[i+time_step,0])
  return np.array(dataX), np.array(dataY)

In [ ]:
time_step = 24

X_train, Y_train = create_dataset(train_data,time_step)
X_valid, Y_valid = create_dataset(valid_data,time_step)

In [ ]:
class myCallback(tf.keras.callbacks.Callback):

  def on_epoch_end(self, epoch, logs={}):

    if(logs.get('mae') < 0.1 ):  


      print("\nMAE < 10%")

      self.model.stop_training = True

callbacks90 = myCallback()

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.LSTM(60, return_sequences=True),
  tf.keras.layers.LSTM(60),
  tf.keras.layers.Dense(30, activation="relu"),
  tf.keras.layers.Dense(10, activation="relu"),
  tf.keras.layers.Dense(1),
])



In [ ]:
optimizer = tf.keras.optimizers.SGD(lr=1.0000e-04, momentum=0.9)
model.compile(loss=tf.keras.losses.Huber(),
              optimizer=optimizer,
              metrics=["mae"])
history = model.fit(train_set,
                    epochs=100,
                     callbacks=[
                        tf.keras.callbacks.EarlyStopping(
                        monitor='val_loss',
                        patience=5,
                        restore_best_weights=True
                        ), myCallback]
                        )